**Задание**

Для каждой страны необходимо подсчитать количество дней, когда количество умерших было больше, чем в среднем по миру за все время наблюдений. Ответом к задаче будет топ-10 стран с наибольшим показателем количества таких дней.

Важные моменты
* Считаются дни, когда было строго больше среднего
* Разные регионы страны вначале необходимо совместить, чтобы статистика считалась верно (смотри последний запрос выше)
* Доработка данных уже после запроса на Python вполне возможна, если это требуется
* Не обязательно делать ровно один агрегационный запрос - можно сделать серию запросов. Главное, чтобы по итогу был подсчитан запрашиваемый набор данных

Результатом должен быть файл `result.json` в следующем формате:
```
{
    "<country 1>": "<number of days>",
    "<country 2>": "<number of days>",
    ...
    "<country 10>": "<number of days>"
}
```

In [3]:
from pymongo import MongoClient
import json
import pandas as pd
client = MongoClient('localhost', 27017)
world = client.covid.world  # коллекция world внутри базы covid

# Функция, чтобы красиво печатать результаты:

def p(x):
    from pprint import pprint
    pprint(list(x))
    
def df(x):
    return pd.DataFrame(list(x))

In [12]:
df(world.aggregate([
    {  # Важно вначале сгрупировать по странам, чтобы среднее по стране считалось честно
        '$group': {
            "_id": {
                "country": "$country",
                "date": "$date"  # группируем сразу по двум полям
            },
            "deaths": {'$sum': "$deaths"},  # для каждой группы складыаем все показатели 
        }
    },
    {
        '$group': {
            "_id": None,
            "avg_deaths": {"$avg": "$deaths"},
            "std_deaths": {"$stdDevPop": "$deaths"}
        }
    }
]))

,_id,avg_deaths,std_deaths
0,None,1027.956109,6531.177153


In [60]:
result = df(world.aggregate([
    {  # Важно вначале сгрупировать по странам, чтобы среднее по стране считалось честно
        '$group': {
            "_id": {
                "country": "$country",
                "date": "$date"  # группируем сразу по двум полям
            },
            "deaths": {'$sum': "$deaths"},  # для каждой группы складыаем все показатели 
        }
    },
    {
        "$match": {
            "deaths": {"$gt": 1027.956109},
        }
    },
    {
        "$group": {
            "_id": {
                "country": "$_id.country"  # ВАЖНО - date лежит теперь внутри _id. Доступ к нему идет через точку
            },
            "count_days": {"$sum": 1}
        }
    },    
    {
        "$sort": {'count_days': -1} # -1 - в порядке убывания, 1 - в порядке возрастания
    }
]))
result._id = pd.DataFrame(result._id.values.tolist()).country
answer = result.head(10)
answer

,_id,count_days
0,China,151
1,Italy,120
2,Iran,115
3,Spain,113
4,France,109
5,US,108
6,United Kingdom,106
7,Netherlands,102
8,Germany,100
9,Belgium,99


In [73]:
answer = answer.set_index('_id').to_dict()['count_days']
answer

{'China': 151,
 'Italy': 120,
 'Iran': 115,
 'Spain': 113,
 'France': 109,
 'US': 108,
 'United Kingdom': 106,
 'Netherlands': 102,
 'Germany': 100,
 'Belgium': 99}

In [74]:
# Пример

with open("result.json", 'w') as f:
    f.write(
        json.dumps(answer)
    )

In [75]:
! cat result.json

{"China": 151, "Italy": 120, "Iran": 115, "Spain": 113, "France": 109, "US": 108, "United Kingdom": 106, "Netherlands": 102, "Germany": 100, "Belgium": 99}

Более подробную информацию про составление агрегационных пайплайнов в MongoDB можно найти в [официальной документации](https://docs.mongodb.com/manual/core/aggregation-pipeline/).